In [4]:
# For reading data sets from the web.
import pandas
# For lots of great things.
import numpy as np
# To make our plots.
import matplotlib.pyplot as plt
%matplotlib inline
# Because sympy and LaTeX make
# everything look wonderful!
from sympy import *
init_printing(use_latex=True)
from IPython.display import display
# We will use this to check our implementation...
from sklearn.decomposition import PCA
# We will grab another data set using Keras
# after we finish up with Iris...
import keras
# Need this for LabelEncoder
from sklearn import preprocessing

In [194]:
student_data = np.array(pandas.read_table("./student-por.csv",
delimiter=";", header=0))

In [195]:
#display(student_data.shape)

In [196]:
le = preprocessing.LabelEncoder()

# Encoding non-numeric data to integers

In [197]:
student_data[0,:]

array(['GP', 'F', 18, 'U', 'GT3', 'A', 4, 4, 'at_home', 'teacher',
       'course', 'mother', 2, 2, 0, 'yes', 'no', 'no', 'no', 'yes', 'yes',
       'no', 'no', 4, 3, 4, 1, 1, 3, 4, 0, 11, 11], dtype=object)

In [198]:
indices = np.array([0,1,3,4,5,8,9,10,11,15,16,17,18,19,20,21,22])
#display(indices)
#le = preprocessing.LabelEncoder()
for i in range(len(indices)):
    j = indices[i]
    le.fit(student_data[:,j])
    student_data[:,j] = le.transform(student_data[:,j])

In [199]:
student_data[0,:]

array([0, 0, 18, 1, 0, 0, 4, 4, 0, 4, 0, 1, 2, 2, 0, 1, 0, 0, 0, 1, 1, 0,
       0, 4, 3, 4, 1, 1, 3, 4, 0, 11, 11], dtype=object)

# Encoding 0's to -1 for binomial data.

### Surely there's a one line method to do this... right, Keras?

In [200]:
indices = np.array([0,1,3,4,5,15,16,17,18,19,20,21,22])
for i in range(len(indices)):
    j = indices[i]
    feature = student_data[:,j]
    feature = np.where(feature==0, -1, feature)
    student_data[:,j] = feature

In [201]:
student_data[0,:]

array([-1, -1, 18, 1, -1, -1, 4, 4, 0, 4, 0, 1, 2, 2, 0, 1, -1, -1, -1, 1,
       1, -1, -1, 4, 3, 4, 1, 1, 3, 4, 0, 11, 11], dtype=object)

# Standardizing the nominal and numerical data.

In [202]:
scaler = preprocessing.StandardScaler()

### I don't think this warning is THAT concerning

In [203]:
temp = student_data[:,[2,14,29,30,31]]
Standardized = scaler.fit_transform(temp)

/nfshome/apps/python-3.6.7/lib/python3.6/site-packages/sklearn/utils/validation.py:590: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/nfshome/apps/python-3.6.7/lib/python3.6/site-packages/sklearn/utils/validation.py:590: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


### Not sure about the -0.0 for the mean...

In [204]:
print('Mean:', round(Standardized.mean()))
print('Standard deviation:', Standardized.std())

Mean: 0.0
Standard deviation: 1.0


In [205]:
student_data[:,[2,14,29,30,31]] = Standardized

## 1-of-C encode nominal data

In [206]:
# Mother's job
## Teacher, Health Care, Civil Services, At Home, Other

# index of current feature
index = 8

# unique values this feature can inhibit
uniqueVals = Matrix(np.unique(student_data[:,index])).T
#display(uniqueVals)

# convert numerical data into categories
shape = uniqueVals.shape[1]
categoricalArr = keras.utils.to_categorical(student_data[:,index], shape)
for i in range (649):
    student_data[i,index] = [categoricalArr[i,:]]
#display(student_data[:,index].shape)
display(student_data[:5,index])

array([list([array([1., 0., 0., 0., 0.], dtype=float32)]),
       list([array([1., 0., 0., 0., 0.], dtype=float32)]),
       list([array([1., 0., 0., 0., 0.], dtype=float32)]),
       list([array([0., 1., 0., 0., 0.], dtype=float32)]),
       list([array([0., 0., 1., 0., 0.], dtype=float32)])], dtype=object)

In [207]:
indices = np.array([9,10,11])

for i in range(len(indices)):
    index = indices[i]
    uniqueVals = Matrix(np.unique(student_data[:,index])).T
    shape = uniqueVals.shape[1]
    categoricalArr = keras.utils.to_categorical(student_data[:,index], shape)
    for i in range (649):
        student_data[i,index] = [categoricalArr[i,:]]

# Father's Job
## Teacher, Health Care, Civil Services, At Home, Other


In [211]:
student_data[283,11]

[array([0., 0., 1.], dtype=float32)]

# Thermometer Encode Nominal Data

In [158]:
# Mother's Education
## None, Primary (4th grade), 5th to 9th grade, Secondary, Higher
index = 6
display(student_data[0:5,index])

array([4, 1, 1, 4, 3], dtype=object)

In [159]:
uniqueVals = Matrix(np.unique(student_data[:,index])).T
#display(uniqueVals)
shape = uniqueVals.shape[1]

In [160]:
# https://stackoverflow.com/questions/49080613/numpy-thermometer-encoding
thermArr = np.array([student_data[:,index]], dtype=np.uint8)
thermArr = np.fliplr(np.unpackbits((1 << thermArr) - 1).reshape(-1,8))
display(thermArr[0:5,:])
display(thermArr[0:5,:shape-1])

array([[1, 1, 1, 1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 1, 1, 0, 0, 0, 0],
       [1, 1, 1, 0, 0, 0, 0, 0]], dtype=uint8)

array([[1, 1, 1, 1],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 1, 1, 1],
       [1, 1, 1, 0]], dtype=uint8)

In [161]:
for i in range (649):
    student_data[i,index] = [thermArr[i,:shape-1]]
display(student_data[0:5,index])

array([list([array([1, 1, 1, 1], dtype=uint8)]),
       list([array([1, 0, 0, 0], dtype=uint8)]),
       list([array([1, 0, 0, 0], dtype=uint8)]),
       list([array([1, 1, 1, 1], dtype=uint8)]),
       list([array([1, 1, 1, 0], dtype=uint8)])], dtype=object)

In [162]:
# Father's Education
## None, Primary (4th grade), 5th to 9th grade, Secondary, Higher
indices = np.array([7,12,13,14,24,25,26,27,28])

for i in range(len(indices)):
    index = indices[i]
    #display(student_data[0:5,index])
    uniqueVals = Matrix(np.unique(student_data[:,index])).T
    shape = uniqueVals.shape[1]
    thermArr = np.array([student_data[:,index]], dtype=np.uint8)
    thermArr = np.fliplr(np.unpackbits((1 << thermArr) - 1).reshape(-1,8))
    for i in range (649):
        student_data[i,index] = [thermArr[i,:shape-1]]

display(student_data[0:5,index])

array([list([array([1, 1, 1, 0], dtype=uint8)]),
       list([array([1, 1, 1, 0], dtype=uint8)]),
       list([array([1, 1, 1, 0], dtype=uint8)]),
       list([array([1, 1, 1, 1], dtype=uint8)]),
       list([array([1, 1, 1, 1], dtype=uint8)])], dtype=object)

In [165]:
display(student_data[0:5,13])

array([list([array([1, 1, 0], dtype=uint8)]),
       list([array([1, 1, 0], dtype=uint8)]),
       list([array([1, 1, 0], dtype=uint8)]),
       list([array([1, 1, 1], dtype=uint8)]),
       list([array([1, 1, 0], dtype=uint8)])], dtype=object)

### Convert results to one-hot encoding
#### the actual output vector doesn't include at least one example of each possible grade so I made a vector of all possibilities for to_categorical

In [166]:
results = student_data[:,32] # output vector
possible_results = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19]

In [167]:
y = keras.utils.to_categorical(results, len(possible_results))

x = np.array(student_data[:,0:32])
#y.shape

In [168]:
x

array([[-1, -1, 1.0316951635937113, ..., 0.0734333641512146,
        -4.155470294933876, -0.1958196099225682],
       [-1, -1, 0.210136683627676, ..., -0.35786304611248015,
        -0.874569782267105, -0.1958196099225682],
       [-1, -1, -1.4329802763043944, ..., 0.5047297744149093,
        0.21906372195515192, 0.4911367513733601],
       ...,
       [1, -1, 1.0316951635937113, ..., 0.5047297744149093,
        -0.14548077945226706, 0.14765857072539595],
       [1, 1, 0.210136683627676, ..., 0.5047297744149093,
        -0.510025280859686, -0.5392977905705323],
       [1, 1, 1.0316951635937113, ..., 0.0734333641512146,
        -0.510025280859686, -0.1958196099225682]], dtype=object)

In [169]:
y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

# Data is now preprocessed

### Note: columns 31 and 32 are not used. I do not believe they are useful for our net.

In [170]:
model = keras.Sequential()

In [171]:
input_size = x.shape[1]
output_size = y.shape[1]

In [172]:
display(input_size)
display(output_size)

OSError: [Errno 12] Cannot allocate memory

32

OSError: [Errno 12] Cannot allocate memory

20

In [173]:
# create the hidden layer
model.add(keras.layers.Dense(400,input_dim=input_size,activation='relu'))

In [174]:
model.add(keras.layers.Dense(200,activation='relu'))

In [175]:
# output layer
model.add(keras.layers.Dense(output_size,activation='softmax'))

In [176]:
# compile
model.compile(loss='categorical_crossentropy',optimizer='adam', metrics=['accuracy'])

In [177]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 400)               13200     
_________________________________________________________________
dense_9 (Dense)              (None, 200)               80200     
_________________________________________________________________
dense_10 (Dense)             (None, 20)                4020      
Total params: 97,420
Trainable params: 97,420
Non-trainable params: 0
_________________________________________________________________


# Model Training

# Austin

In [212]:
x

array([[-1, -1, 1.0316951635937113, ..., 0.0734333641512146,
        -4.155470294933876, -0.1958196099225682],
       [-1, -1, 0.210136683627676, ..., -0.35786304611248015,
        -0.874569782267105, -0.1958196099225682],
       [-1, -1, -1.4329802763043944, ..., 0.5047297744149093,
        0.21906372195515192, 0.4911367513733601],
       ...,
       [1, -1, 1.0316951635937113, ..., 0.5047297744149093,
        -0.14548077945226706, 0.14765857072539595],
       [1, 1, 0.210136683627676, ..., 0.5047297744149093,
        -0.510025280859686, -0.5392977905705323],
       [1, 1, 1.0316951635937113, ..., 0.0734333641512146,
        -0.510025280859686, -0.1958196099225682]], dtype=object)

In [178]:
x_train = x[0:518,:]
x_test = x[519:649,:]

y_train = y[0:518,:]
y_test = y[519:649,:]

history = model.fit(x,y,batch_size = 32, epochs = 7, verbose = 0, validation_split = 0.2)
#history = model.fit(x,y,batch_size = 32, epochs = 160, verbose = 0, validation_split = 0.25)

ValueError: setting an array element with a sequence.

In [ ]:
#plt.plot(history.history['loss'])
#plt.title('model loss')
#plt.ylabel('loss')
#plt.xlabel('epoch')
#plt.legend(['train'], loc='upper left')
#plt.show()


In [ ]:
#score = model.evaluate(x,y, verbose=0)

score = model.evaluate(x_test,y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
plt.figure(1)

# summarize history for accuracy

plt.subplot(211)
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train','test'], loc ='upper left')

# summarize history for loss

plt.subplot(212)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train','test'], loc ='upper left')

plt.tight_layout()
plt.show()

In [ ]:
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
## PCAs of Data

In [ ]:
#def func(X):
 #   return X
#xFunc = np.apply_along_axis(func,0,x)


#U,S,V = np.linalg.svd(xFunc,full_matrices=True)

# Plot the percent of variance accounted for by each feature
#plt.plot(100.0*S/np.sum(S))
#plt.ylabel("Percent Variance")
#plt.xlabel("Singular Value")
#plt.show()

In [ ]:
# https://www.youtube.com/watch?v=Lsue2gEM9D0
pca=PCA()
pca.fit(x)
pca_data = pca.transform(x)

In [ ]:
per_var = np.round(pca.explained_variance_ratio_* 100, decimals = 1)
labels = [str(i) for i in range (1, len(per_var)+1)]

plt.bar(x=range(1,len(per_var)+1), height=per_var, tick_label=labels)
plt.ylabel("% var")
plt.xlabel("PC")
plt.title("Scree Plot")
plt.show()

In [ ]:
display(x.shape[1])
display(y.shape[1])